In [1]:
import pandas as pd
import numpy as np
import pycountry as pyc
import regex as re
from fuzzywuzzy import fuzz
from collections import defaultdict
from tqdm import tqdm

In [2]:
raw = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-05-18/survey.csv')

In [8]:
raw['job_title'].unique().shape

(12622,)

## Eliminate (Almost) Empty Columns and rows without industry, education, gender, and race

In [30]:
nullity = raw.isna().sum()/raw.shape[0]
nullity

timestamp                                   0.000000
how_old_are_you                             0.000000
industry                                    0.002249
job_title                                   0.000000
additional_context_on_job_title             0.742185
annual_salary                               0.000000
other_monetary_comp                         0.258654
currency                                    0.000000
currency_other                              0.993672
additional_context_on_income                0.890515
country                                     0.000000
state                                       0.178675
city                                        0.002973
overall_years_of_professional_experience    0.000000
years_of_experience_in_field                0.000000
highest_level_of_education_completed        0.007472
gender                                      0.005642
race                                        0.005566
dtype: float64

In [31]:
dropped_cols = list(nullity[nullity>.7].index)
cleaned = raw.drop(columns=dropped_cols + ['timestamp', 'gender', 'race'])
cleaned.drop(cleaned.index[(cleaned['industry'].isna()) | (cleaned['gender'].isna()) | (cleaned['race'].isna()) | (cleaned['highest_level_of_education_completed'].isna())], inplace=True)

## Fixing Country of Origin

In [32]:
cleaned['country'].value_counts()

United States     8894
USA               7775
US                2427
Canada            1515
United Kingdom     572
                  ... 
Sri lanka            1
Kuwait               1
Bangladesh           1
Eritrea              1
france               1
Name: country, Length: 291, dtype: int64

In [33]:
def find_country_name(inp):
    inp = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", inp)
    if pyc.countries.get(name=inp):
        return pyc.countries.get(name=inp).alpha_3
    elif pyc.countries.get(alpha_2=inp):
        return pyc.countries.get(alpha_2=inp).alpha_3
    elif pyc.countries.get(alpha_3=inp):
        return inp.upper() 
    elif 'england' in inp.lower() or 'scotland' in inp.lower() or 'uk' in inp.lower():
        return 'GBR'
    elif 'usa' in inp.lower() or 'america' in inp.lower() or 'us' in inp.lower() or 'united states' in inp.lower() or 'hartford' == inp.lower() or 'san francisco' == inp.lower():
        return 'USA'
    elif fuzz.ratio('united states', inp.lower()) > 70:
        return 'USA'
    elif 'remote' in inp.lower():
        return 'remote'
    elif fuzz.ratio('united kingdom', inp.lower()) > 70 or fuzz.ratio('england', inp.lower()) > 70:
        return 'GBR'
    elif 'united kingdom' in inp.lower() or 'jersey' in inp.lower():
        return 'GBR'
    elif fuzz.ratio('canada', inp.lower()) > 70 or 'canada' in inp.lower():
        return 'CAD'
    elif 'italy' in inp.lower():
        return pyc.countries.get(name='italy').alpha_3
    elif 'new zealand' in inp.lower():
        return pyc.countries.search_fuzzy('new zealand')[0].alpha_3
    elif 'catalonia' in inp.lower():
        return pyc.countries.get(name='spain').alpha_3
    elif 'neder' in inp.lower():
        return pyc.countries.get(name='Netherlands').alpha_3
    elif 'luxemb' in inp.lower():
        return pyc.countries.get(name='luxembourg').alpha_3
    elif inp == 'UAE':
        return 'ARE'
    else:
        try:
            return pyc.countries.search_fuzzy(inp)[0].alpha_3
        except:
            return np.NAN

In [34]:

inp = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", 'Usa')
pyc.countries.get(alpha_3=inp)

Country(alpha_2='US', alpha_3='USA', flag='🇺🇸', name='United States', numeric='840', official_name='United States of America')

In [35]:
cleaned['nation'] = cleaned['country'].apply(find_country_name)
cleaned.loc[cleaned['country'] == r'$2,175.84/year is deducted for benefits', 'nation'] = 'CAD'
cleaned.loc[cleaned['country'].str.contains('I was brought in'), 'nation'] = 'USA'
cleaned.loc[cleaned['country'].str.contains('I earn commission'), 'nation'] = 'USA'
cleaned.loc[cleaned['country'] == 'Currently finance', 'nation'] = 'USA'
cleaned.loc[cleaned['country'] == 'Contracts', 'nation'] = 'USA'
cleaned.loc[cleaned['country'] == 'Global', 'nation'] = 'CAD'
cleaned.loc[cleaned['country'] == 'International', 'nation'] = 'USA'
cleaned.loc[cleaned['country'] == 'UXZ', 'nation'] = 'USA'
cleaned.loc[cleaned['country'] == 'United y', 'nation'] = 'USA'
cleaned.loc[cleaned['country'] == 'europe', 'nation'] = 'CZE'
cleaned.drop(columns=['country'], inplace=True)

In [36]:
cleaned.drop(cleaned.index[(cleaned['city'].isna()) & (cleaned['state'].isna())], inplace=True)

## Fixing Compensation for Currency Exchange Rates

In [37]:
cleaned['total_compensation'] = cleaned['annual_salary'] + cleaned['other_monetary_comp'].fillna(0)
cleaned.drop(columns=['annual_salary', 'other_monetary_comp'], inplace=True)

In [38]:
cleaned['currency'].value_counts()

USD        21478
CAD         1529
GBP         1487
EUR          560
AUD/NZD      459
Other        129
CHF           35
SEK           32
JPY           21
ZAR           13
HKD            4
Name: currency, dtype: int64

In [39]:
cleaned.drop(cleaned.index[cleaned['currency'] == 'Other'], inplace=True)

In [40]:
def aud_nzd(series):
    if series['nation'] == 'AUS':
        return 'AUD'
    if series['nation'] == 'NZL':
        return 'NZD'
    else:
        return series['currency'] 

In [41]:
cleaned['currency'] = cleaned.apply(aud_nzd, axis=1)
cleaned.drop(cleaned.index[cleaned['currency']=='AUD/NZD'], inplace=True)

In [42]:
# Data from finance.yahoo.com for May 1st, 2021
rates = {'USD': 1, 'CAD': 1/1.2078, 'GBP': 1.4180, 'EUR': 1.2190, 'AUD': 1/4.3500, 'NZD': .7243, 'CHF': .9001, 'SEK': 1/8.3094, 'JPY': 1/109.9070, 'ZAR': .069, 'HKD': .13}
def get_USD_std(rates, series):
    return rates[series['currency']]*series['total_compensation']

In [43]:
cleaned['usd_compensation'] = cleaned.apply(lambda x: get_USD_std(rates, x), axis=1)
cleaned.drop(columns=['total_compensation'], inplace=True)

In [44]:
cleaned.loc[cleaned.usd_compensation > 5000000]

,how_old_are_you,industry,job_title,currency,state,city,overall_years_of_professional_experience,years_of_experience_in_field,highest_level_of_education_completed,gender,race,nation,usd_compensation
3605,25-34,Utilities & Telecommunications,Operations Manager,USD,NaN,Bogota,8 - 10 years,1 year or less,Some college,Man,"Hispanic, Latino, or Spanish origin",COL,102000000.0


## Gender

In [45]:
def get_gender(gender):
    if gender == 'Man':
        return 'M'
    if gender == 'Woman':
        return 'F'
    else:
        return 'Other'
cleaned['gender'] = cleaned['gender'].apply(get_gender)

## Race

In [46]:
def get_race(race):
    if ',' in race:
        return race.split(',')[0]
    return race
cleaned['race'] = cleaned['race'].apply(get_race)

In [47]:
cleaned['race'].value_counts()

White                                                     21644
Asian or Asian American                                    1554
Hispanic                                                    876
Black or African American                                   770
Another option not listed here or prefer not to answer      537
Middle Eastern or Northern African                          131
Native American or Alaska Native                            104
Name: race, dtype: int64

## Industry

In [48]:
def make_buckets(df, col, threshold, partial=False):
    buckets = defaultdict(list)
    word_bank = df[col].unique()
    taken = set()
    with tqdm(total=len(word_bank)) as pbar:
        for word1 in word_bank:
            if word1 in taken:
                continue
            for word2 in word_bank: 
                if word2 in taken:
                    continue
                if partial:
                    if (fuzz.ratio(word1, word2) > threshold or fuzz.partial_token_set_ratio(word1, word2) > threshold) and word1!=word2 and word2 not in taken:
                        buckets[word1].append(word2)
                        taken.add(word1)
                        taken.add(word2)
                else:
                    if fuzz.ratio(word1, word2) > threshold and word1!=word2 and word2 not in taken:
                        buckets[word1].append(word2)
                        taken.add(word1)
                        taken.add(word2)
            pbar.update(1)
    return buckets

def replace_names(inp_name, bucket, replace_list):
    '''
    inp_name: input with apply
    bucket: group leader
    replace_list: similar words to be replaced
    '''
    if inp_name in replace_list:
        return bucket 
    else:
        return inp_name

In [49]:
for word, copy_list in make_buckets(cleaned, 'industry', 70, partial=True).items():
    cleaned['industry'] = cleaned['industry'].apply(lambda x: replace_names(x, word, copy_list))

 16%|█▌        | 169/1057 [00:00<00:04, 185.06it/s]


## Title

In [50]:
for word, copy_list in make_buckets(cleaned, 'job_title', 90).items():
    cleaned['job_title'] = cleaned['job_title'].apply(lambda x: replace_names(x, word, copy_list))

 86%|████████▌ | 10675/12398 [03:32<00:34, 50.30it/s]


## Final Export

In [51]:
cleaned.isna().sum()

how_old_are_you                                0
industry                                       0
job_title                                      0
currency                                       0
state                                       4429
city                                          55
overall_years_of_professional_experience       0
years_of_experience_in_field                   0
highest_level_of_education_completed           0
gender                                         0
race                                           0
nation                                         0
usd_compensation                               0
dtype: int64

In [52]:
cleaned.head(3)

,how_old_are_you,industry,job_title,currency,state,city,overall_years_of_professional_experience,years_of_experience_in_field,highest_level_of_education_completed,gender,race,nation,usd_compensation
0,25-34,Education (Higher Education),Research and Instruction Librarian,USD,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,F,White,USA,55000.0
1,25-34,Computing or Tech,Change & Internal Communications Manager,GBP,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Other,White,GBR,83094.8
2,25-34,"Accounting, Banking & Finance",Marketing Specialist,USD,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,F,White,USA,34000.0


In [53]:
final_cleaned = pd.DataFrame()
final_cleaned['industry'] = cleaned['industry']
final_cleaned['age_group'] = cleaned['how_old_are_you']
final_cleaned['title'] = cleaned['job_title']
final_cleaned['pay'] = cleaned['usd_compensation']
final_cleaned['country'] = cleaned['nation']
final_cleaned['state'] = cleaned['state']
final_cleaned['city'] = cleaned['city']
final_cleaned['total_experience'] = cleaned['overall_years_of_professional_experience']
final_cleaned['field_experience'] = cleaned['years_of_experience_in_field']
final_cleaned['education'] = cleaned['highest_level_of_education_completed']
final_cleaned['gender'] = cleaned['gender']
final_cleaned['race'] = cleaned['race']
final_cleaned.to_csv('salary_metadata.csv')
final_cleaned.head()

,industry,age_group,title,pay,country,state,city,total_experience,field_experience,education,gender,race
0,Education (Higher Education),25-34,Research and Instruction Librarian,55000.0,USA,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,F,White
1,Computing or Tech,25-34,Change & Internal Communications Manager,83094.8,GBR,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Other,White
2,"Accounting, Banking & Finance",25-34,Marketing Specialist,34000.0,USA,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,F,White
3,Nonprofits,25-34,Program Manager,65000.0,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,F,White
4,"Accounting, Banking & Finance",25-34,Accounting Manager,67000.0,USA,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,F,White
